In [1]:
%pip install openai
from pprint import pprint
import openai
from openai import OpenAI
import os

Note: you may need to restart the kernel to use updated packages.


In [11]:
#openai.api_key = os.environ.get('OPENAI_API_KEY')
# read the api key from a file
import json
with open('openai_key.json', 'r') as f:
    openai.api_key = json.load(f)['OPENAI_API_KEY']


In [12]:
client = OpenAI(api_key=openai.api_key)    

In [13]:
#read in credit_risk_dataset.csv and chunk it into 1000 row chunks
import pandas as pd
df = pd.read_csv('credit_risk_dataset.csv')
#chunk into 1000 row chunks
df_chunks = [df[i:i+1000] for i in range(0,df.shape[0],1000)]
# write out chucks to separate json files
for i in range(len(df_chunks)):
    df_chunks[i].to_json(f'credit_risk_dataset_{i}.json', orient='records')

In [14]:
file = client.files.create(file=open('credit_risk_dataset_0.json', 'rb'), purpose='assistants')
file_id = file.id

In [15]:
print(file.id)
file_list = client.files.list()
print(file_list)

file-GY5abxO8Bg9CrPAYMCMpzNsO
SyncPage[FileObject](data=[FileObject(id='file-GY5abxO8Bg9CrPAYMCMpzNsO', bytes=285278, created_at=1700158204, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-3GhP01ghbawE6cqlKsF2kCFo', bytes=1804682, created_at=1699793153, filename='credit_risk_dataset.csv', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-pgryNtOQWnpV6okWteSzh8aQ', bytes=285278, created_at=1699791056, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-CuKkMFLdepwWXr7RWHyTRU35', bytes=285278, created_at=1699742998, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-GyM5HYkPSL20RcSWEogdXv5P', bytes=285278, created_at=1699742476, filename='credit_risk_dataset_0.json', object='file',

In [ ]:
get_youngsters = {
  "name": "get_youngsters",
  "description": "Get the youngsters",
  "parameters": {
    "type": "object",
    "properties": {
      "age": {
        "type": "integer",
        "description": "Age cutoff"
      }
    },
    "required": [
      "age"
    ]
  }
}

In [16]:
assistant = client.beta.assistants.create(
    instructions="This is a test assistant",
    model = "gpt-4-1106-preview",
    tools = [{"type":"retrieval"}],
    file_ids = [file_id]
)
print(assistant.id)

asst_Fb1LNvo5NNGFYVyAlRuq17Um


In [17]:
# update assistant if you need to change the assistant
assistant = client.beta.assistants.update(
    assistant.id,
    instructions="This is a test assistant",
    model = "gpt-4-1106-preview",
    tools = [{"type":"retrieval"}],
    file_ids = [file_id]
)

In [18]:
# list assistants
assistants = client.beta.assistants.list(order="desc")

In [20]:
# lets suppose you want to use and existing assistant
assistant_id = 'asst_Fb1LNvo5NNGFYVyAlRuq17Um'
assistant = client.beta.assistants.retrieve(assistant_id)


In [21]:
# create a thread
thread = client.beta.threads.create()
print(thread.id)

thread_ry20wgVVkAU6vWnAryQUunEB


In [22]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content="Hello, I'm a message"
)

In [23]:
# run the assistant to get a response
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "I want to buy a house",
)
print(run.id)

run_yV5gHQOYGR1s1oURrBr5JpQf


In [24]:
# retrieve the response
status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)
print(status)

Run(id='run_yV5gHQOYGR1s1oURrBr5JpQf', assistant_id='asst_Fb1LNvo5NNGFYVyAlRuq17Um', cancelled_at=None, completed_at=1700158285, created_at=1700158283, expires_at=None, failed_at=None, file_ids=['file-GY5abxO8Bg9CrPAYMCMpzNsO'], instructions='I want to buy a house', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1700158283, status='completed', thread_id='thread_ry20wgVVkAU6vWnAryQUunEB', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [25]:
# list out run steps
steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)
print(steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_42LZo3CTcyljCNDe8MmQwDAh', assistant_id='asst_Fb1LNvo5NNGFYVyAlRuq17Um', cancelled_at=None, completed_at=1700158285, created_at=1700158283, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_yV5gHQOYGR1s1oURrBr5JpQf', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_qQB2DKN5LR1azJvls5hJy0X3'), type='message_creation'), thread_id='thread_ry20wgVVkAU6vWnAryQUunEB', type='message_creation', expires_at=None)], object='list', first_id='step_42LZo3CTcyljCNDe8MmQwDAh', last_id='step_42LZo3CTcyljCNDe8MmQwDAh', has_more=False)


In [ ]:
# CANCEL A RUN - Dont do this if you want to use the run
run = client.beta.threads.runs.cancel(
    thread_id = thread.id,
    run_id = run.id
)
print(run)

In [39]:
# loop to see if run is complete
import time
while True:
    start = time.time()
    status = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    if status.status == 'complete':
        break
    time.sleep(10)
    if time.time() - start > 300:   # 5 minutes
        break
print(status)

KeyboardInterrupt: 

In [41]:
# get all the messages
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
for message in messages:
    print(message.content)

[MessageContentText(text=Text(annotations=[], value="Hello! It looks like you're interested in buying a house and have uploaded a file that might be related to this process. How can I assist you with it today? If you need information extracted or summarized from the file, please let me know what you're looking for specifically."), type='text')]
[MessageContentText(text=Text(annotations=[], value="Hello, I'm a message"), type='text')]


In [ ]:
# clean up the assistant
assistant = client.beta.assistants.delete(assistant.id)
print(assistant)
